This notebook provides an example of running through and making a bunch of DOI updates based on records in ScienceBase. In this case, we had previously reserved DOIs, all pointing at ScienceBase Items as their de-referencing URLs, and we now need to finalize the DOIs and turn them on for real. It works with Brandon Serna's usgs_datatools package, where the latest version works with the new DOI REST API. Most of the parts and pieces of this should be fairly easily reused by others needing to do something similar.

The first few blocks here do all the same things Brandon has shown in other examples, setting up a session with the DOI tool to do work.

In [1]:
import os
import json
import requests
import getpass
from IPython.display import display

from usgs_datatools import doi

In [26]:
username = input('User: ')
password = getpass.getpass('USGS AD Password: ')

User: bserna@usgs.gov
USGS AD Password: ········


#### DOI Session

In [27]:
doi_session = doi.DoiSession(env='production')
doi_session.doi_authenticate(username, password)

In this case, I'm starting from the perspective of the existing ScienceBase Items where we previously recorded the reserved DOIs as one of the identifiers. I need to validate that all the critical information associated with that DOI matches what's in the ScienceBase Item, resetting a couple of attributes along the way and making the DOI public. I could just run it all in one process that goes through each ScienceBase Item, connects to the DOI tool, and does the work, but since we are running this in a Notebook environment, we can simply build out a stripped down data object in memory and then run against that.

We're dealing with two different ScienceBase collections with 1,719 items in each one, so we need to paginate through ScienceBase search results to build out what we need. You could also do this with pysb and would need to use something like that if dealing with restricted items. Because we just finished the data release process on these and made the collections public, we can simply access the ScienceBase REST API with requests.

In [5]:
sbItems = []
for collectionID in ['527d0a83e4b0850ea0518326', '5951527de4b062508e3b1e79']:

    nextLink = 'https://www.sciencebase.gov/catalog/items?max=100&format=json&fields=title,identifiers&parentId='+collectionID

    while nextLink is not None:
        if nextLink is not None:
            sbResult = requests.get(nextLink).json()

            if 'nextlink' in sbResult.keys():
                nextLink = sbResult['nextlink']['url']
            else:
                nextLink = None

            if len(sbResult['items']) != 0:
                for item in sbResult['items']:
                    thisSBItem = {'id':item['id']}
                    thisSBItem['title'] = item['title']
                    thisSBItem['doi'] = next((i['key'] for i in item["identifiers"] if i['type'] == 'doi'), None)
                    sbItems.append(thisSBItem)

In [7]:
# See if we pulled the expected number of items - 3,438
print(len(sbItems))

sbItems[0]

3438


{'id': '58fa530ee4b0b7ea545252af',
 'title': 'Philadelphia Vireo (Vireo philadelphicus) bPHVIx_CONUS_2001v1 Habitat Map',
 'doi': 'doi:10.5066/F7K935WV'}

In [36]:
%%time
for index, sbItem in enumerate(sbItems):
    # Put in a break for testing
    #if index > 500: break

    # Get the current record from the DOI database
    thisDOIRecord = doi_session.get_doi(sbItem['doi'])

    # Convert the returned 'message' to a dict for comparing current attributes
    # When the returned message is an error do x
    if len(thisDOIRecord) <= 3:
        print('An error occured: ', thisDOIRecord)
        break
        #thisDOIDict = json.loads(thisDOIRecord['message'])
    
    # The proccess horked out, so I'm throwing in a check on status to see if I can get it to finish off eventually
    if thisDOIRecord['status'] == 'public':
        print(u'\u2713', ' ', thisDOIRecord['doi'])
        continue
    
    # Create a dictionary to conduct the update containing all the information it's supposed to have
    thisDoi = sbItem.copy()
    # Get rid of the ScienceBase ID for the doi update packet (though it doesn't seem to break anything if left in)
    thisDoi.pop('id')

    # Set some hard-coded parameters, including making the DOI public
    thisDoi['status'] = 'public'
    thisDoi['pubDate'] = '2018'
    thisDoi['ipdsNumbers'] = [{'ipdsNumber': '082267', 'ipdsType': 'DATA_RELEASE'}]
    #thisDoi['dataSourceId'] = 59507
    thisDoi['dataSourceName'] = 'Core Science Analytics, Synthesis and Libraries'
    # Despite including the IPDS number, I'm getting an error returned when trying to make public
    # I'm setting this flag to see if I can get the process done, since there seems to be a checking error
    thisDoi['noPublicationIdAvailable'] = True
    
    # Check to make sure the DOI URL matches the ScienceBase Item
    # If not, add it to the update packet
    if thisDOIRecord['url'].split('/')[-1] != sbItem['id']:
        thisDoi['url'] = 'https://www.sciencebase.gov/catalog/item/'+sbItem['id']
    
    # This will be the DOI update package plus the parameter to make it public
    #display(thisDoi)
    
    # Update the DOI with the update packet, including setting to public from reserved
    upd_r = doi_session.doi_update(thisDoi)
    #display(upd_r)
    print(u'\u2713\u2713 ', upd_r['doi'], upd_r['status'], f' line: {index}')
    break

✓   doi:10.5066/F7K935WV
✓   doi:10.5066/F75719FH
✓   doi:10.5066/F7H130D4
✓   doi:10.5066/F7FJ2F43
✓   doi:10.5066/F7H41PR5
✓   doi:10.5066/F7XG9PJP
✓   doi:10.5066/F7HQ3X9V
✓   doi:10.5066/F7ZK5F1W
✓   doi:10.5066/F73B5XGB
✓   doi:10.5066/F7348HP4
✓   doi:10.5066/F7VQ30Z9
✓   doi:10.5066/F77W69G1
✓   doi:10.5066/F7P55KSS
✓   doi:10.5066/F7Z60MG3
✓   doi:10.5066/F75B00VT
✓   doi:10.5066/F7CN72BJ
✓   doi:10.5066/F7JW8C8Q
✓   doi:10.5066/F7ST7N7P
✓   doi:10.5066/F7D798T7
✓   doi:10.5066/F7J101JH
✓   doi:10.5066/F7WQ026F
✓   doi:10.5066/F7N58JQ9
✓   doi:10.5066/F70863MZ
✓   doi:10.5066/F7513WJK
✓   doi:10.5066/F7DV1H84
✓   doi:10.5066/F7RJ4GVG
✓   doi:10.5066/F7668BHF
✓   doi:10.5066/F7765CQQ
✓   doi:10.5066/F7P8496H
✓   doi:10.5066/F71Z42TP
✓   doi:10.5066/F79K48P2
✓   doi:10.5066/F7PC30VZ
✓   doi:10.5066/F7B856D7
✓   doi:10.5066/F75M6409
✓   doi:10.5066/F7D798RB
✓   doi:10.5066/F71Z42RS
✓   doi:10.5066/F75Q4TBB
✓   doi:10.5066/F7SJ1HZD
✓   doi:10.5066/F7Q52N1X
✓   doi:10.5066/F70Z71MD
